In [7]:
import pandas as pd
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import accuracy_score

In [8]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/draft.csv')

In [9]:
X =  df.drop(columns="diagnosis",axis=1)
Y =  df["diagnosis"]

In [10]:
print(X)

       heart rate  ap_hi  ap_lo
0              62    110     80
1              85    140     90
2              64    130     70
3              82    150    100
4              56    100     60
...           ...    ...    ...
66138          80    150     80
66139          76    120     80
66140         105    180     90
66141          72    135     80
66142          72    120     80

[66143 rows x 3 columns]


In [11]:
print(Y)

0        2
1        3
2        2
3        3
4        1
        ..
66138    2
66139    2
66140    3
66141    2
66142    2
Name: diagnosis, Length: 66143, dtype: int64


In [12]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.2, stratify=Y, random_state=2 )

In [13]:
print(X.shape, X_train.shape, X_test.shape)

(66143, 3) (52914, 3) (13229, 3)


In [14]:
model = LogisticRegression(max_iter=1500)

In [15]:
model.fit(X_train, Y_train)

LogisticRegression(max_iter=1500)

In [16]:
y_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(Y_train,y_train_prediction )

In [17]:
print("accuracy on training data:", training_data_accuracy)

accuracy on training data: 0.9822353252447368


In [18]:
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction,Y_test )

In [19]:
print("accuracy on test data:", test_data_accuracy)

accuracy on test data: 0.9820092221634288


In [20]:
input_data=(82,150,100)
input_data_as_numpy_array= np.asarray(input_data)
import_data_reshaped = input_data_as_numpy_array.reshape(1,3) 
prediction= model.predict(import_data_reshaped)
print(prediction)

[3]


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


In [21]:
input_data=(82,150,86)
input_data_as_numpy_array= np.asarray(input_data)
import_data_reshaped = input_data_as_numpy_array.reshape(1,3) 
prediction= model.predict(import_data_reshaped)
print(prediction)

[3]


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


In [26]:
import tensorflow as tf
import numpy as np
from sklearn.linear_model import LogisticRegression

# # some random data to train and test on
# x = np.random.normal(size=(60, 21))
# y = np.random.uniform(size=(60,)) > 0.5

# # fit the sklearn model on the data
# sklearn_model = LogisticRegression().fit(x, y)
sklearn_model = model

# create a TF model with the same architecture
tf_model = tf.keras.models.Sequential()
tf_model.add(tf.keras.Input(shape=(3,)))
tf_model.add(tf.keras.layers.Dense(3))

# assign the parameters from sklearn to the TF model
tf_model.layers[0].weights[0].assign(sklearn_model.coef_.transpose())
tf_model.layers[0].bias.assign(sklearn_model.intercept_)

# Verifying usign your model
input_data=(80,120,87)
input_data_as_numpy_array= np.asarray(input_data)
import_data_reshaped = input_data_as_numpy_array.reshape(1,3) 
# prediction = model.predict(import_data_reshaped)
# print("=====", prediction, '====')

# Verifying uding tf model
output_tf = tf_model(import_data_reshaped)
highest_prob_index = tf.argmax(output_tf, axis=1)
highest_prob_index_list = highest_prob_index.numpy().tolist()
print(highest_prob_index_list[0] + 1 )

2


In [23]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
# # Create a serving function for the model
@tf.function(input_signature=[tf.TensorSpec(shape=[None, 3], dtype=tf.float32)])
def serving_function(x):
    return tf_model(x)

# # Get the concrete function from the serving function
concrete_func = serving_function.get_concrete_function()

# # Convert the TensorFlow model to a TensorFlow Lite model
converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func])
tflite_model = converter.convert()

# # Save the model
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

In [30]:
# testing the tflite model from to see if it works
import numpy as np
import tensorflow as tf

interpreter = tf.lite.Interpreter("/content/model.tflite")

input_data=(80,150,87)
input_data_as_numpy_array= np.asarray(input_data)
import_data_reshaped = input_data_as_numpy_array.reshape(1,3)

output_tf = tf_model(import_data_reshaped)

highest_prob_index = tf.math.argmax(output_tf, axis=1)
highest_prob_index_list = highest_prob_index.numpy().tolist()
print(highest_prob_index_list[0] + 1 )

3
